In [26]:
adj_norm=1
base_model_dump=None
batch_size=10
bilin_q=1
cross_rate=0.1
ctx='cpu'

data_folder='/mnt/sda/wwj/graph_adversarial_attack/dropbox/data'

dataset='pubmed'

del_rate=0
dropout=0.5
er_p=0
feat_dim=0

feature_dim=None

fold=1
frac_meta=0
gm='mean_field'
hidden=0
idx_start=None
latent_dim=64
learning_rate=0.01
logfile=None
max_c=0
max_lv=1
max_n=0
meta_test=0
min_c=0
min_n=0
mlp_hidden=64
mutate_rate=0.2
n_graphs=0
n_hops=3
num_class=None
num_epochs=200
num_instances=None
num_mod=1
num_steps=500000
out_dim=0
phase='train'
population_size=100
rand_att_type=None
reward_type='binary'
rounds=10
save_dir='/home/wwj/scratch/results/del_edge_attack/pubmed-gcn-0.01/rl-lv-1-q-1-meta-0'
saved_model='/mnt/sda/wwj/graph_adversarial_attack/dropbox/scratch/results/node_classification/pubmed/model-gcn-epoch-best-0.01'
seed=1
targeted=0
weight_decay=0.0005

In [18]:
import os
import sys
import numpy as np
import torch
import networkx as nx
import random
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from copy import deepcopy

In [40]:
import os
import sys
import numpy as np
import torch
import networkx as nx
import random
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from copy import deepcopy
# import cPickle as cp
import pickle as pkl

In [46]:
# node_utils
import numpy as np
import pickle as pkl
import networkx as nx
import torch
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
import os

In [71]:
# gcn_modules
import os
import sys
import numpy as np
import torch
import networkx as nx
import random
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
# import cPickle as cp
import pickle as cp

In [22]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [28]:
list(np.loadtxt(data_folder +'/'+ dataset + '/train_idx.txt', dtype=int))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59]

In [41]:
def load_raw_graph(data_folder, dataset_str):
    bin_file = "{}/ind.{}.{}".format(data_folder, dataset_str, 'graph')
    if os.path.isfile(bin_file):
        with open(bin_file, 'rb') as f:
            if sys.version_info > (3, 0):
                graph = pkl.load(f, encoding='latin1')
            else:
                graph = pkl.load(f)
    else:
        txt_file = data_folder + '/adj_list.txt'
        graph = {}
        with open(txt_file, 'r') as f:
            cur_idx = 0
            for row in f:
                row = row.strip().split()
                adjs = []
                for j in range(1, len(row)):
                    adjs.append(int(row[j]))
                graph[cur_idx] = adjs
                cur_idx += 1

    return graph

In [44]:
class StaticGraph(object):
    graph = None

    @staticmethod
    def get_gsize():
        return torch.Size( (len(StaticGraph.graph), len(StaticGraph.graph)) )

In [42]:
def load_txt_data(data_folder, dataset_str):
    idx_train = list(np.loadtxt(data_folder + '/train_idx.txt', dtype=int))
    idx_val = list(np.loadtxt(data_folder + '/val_idx.txt', dtype=int))
    idx_test = list(np.loadtxt(data_folder + '/test_idx.txt', dtype=int))
    labels = np.loadtxt(data_folder + '/label.txt')
    
    with open(data_folder + '/meta.txt', 'r') as f:
        num_nodes, num_class, feature_dim = [int(w) for w in f.readline().strip().split()]

    graph = load_raw_graph(data_folder, dataset_str)
    assert len(graph) == num_nodes
    StaticGraph.graph = nx.from_dict_of_lists(graph)
    
    row_ptr = []
    col_idx = []
    vals = []
    with open(data_folder + '/features.txt', 'r') as f:
        nnz = 0
        for row in f:
            row = row.strip().split()
            row_ptr.append(nnz)            
            for i in range(1, len(row)):
                w = row[i].split(':')
                col_idx.append(int(w[0]))
                vals.append(float(w[1]))
            nnz += int(row[0])
        row_ptr.append(nnz)
    assert len(col_idx) == len(vals) and len(vals) == nnz and len(row_ptr) == num_nodes + 1

    features = sp.csr_matrix((vals, col_idx, row_ptr), shape=(num_nodes, feature_dim))
    
    return preprocess_features(features), labels, idx_train, idx_val, idx_test

In [52]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)    
    sp_tuple = sparse_to_tuple(features)
    idxes = torch.LongTensor(sp_tuple[0]).transpose(0, 1).contiguous()
    values = torch.Tensor(sp_tuple[1].astype(np.float32))

    mat = torch.sparse.FloatTensor(idxes, values, torch.Size(sp_tuple[2]))
    if ctx == 'gpu':
        mat = mat.cuda()
    return mat

In [53]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

In [60]:
features, labels, _, idx_val, idx_test = load_txt_data(data_folder + '/' + dataset, dataset)    

In [61]:
features

tensor(indices=tensor([[    0,     0,     0,  ..., 19716, 19716, 19716],
                       [  394,   384,   382,  ...,    16,     7,     1]]),
       values=tensor([0.0283, 0.0162, 0.0125,  ..., 0.0104, 0.0030, 0.0145]),
       size=(19717, 500), nnz=988031, layout=torch.sparse_coo)

In [62]:
features = Variable( features )
labels = Variable( torch.LongTensor( np.argmax(labels, axis=1) ) )

In [67]:
if ctx == 'gpu':
    labels = labels.cuda()

In [69]:
def load_base_model():
    assert saved_model is not None
    with open('%s-args.pkl' % saved_model, 'rb') as f:
        base_args = pkl.load(f)
    if 'mean_field' in saved_model: # XX
        mod = S2VNodeClassifier
    elif 'gcn' in saved_model: # this 
        mod = GCNModule

    gcn = mod(**vars(base_args))
    if ctx == 'gpu':
        gcn = gcn.cuda()
    gcn.load_state_dict(torch.load(saved_model+ '.model'))
    gcn.eval()
    return gcn

In [72]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)

        weights_init(self)

    def forward(self, input, adj):
        if input.data.is_sparse:
            support = gnn_spmm(input, self.weight)
        else:
            support = torch.mm(input, self.weight)
        
        output = gnn_spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class GCNModule(nn.Module):
    def __init__(self, **kwargs):
        super(GCNModule, self).__init__()
        self.gc1 = GraphConvolution(kwargs['feature_dim'], kwargs['latent_dim'])
        self.gc2 = GraphConvolution(kwargs['latent_dim'], kwargs['num_class'])
        self.dropout_rate = kwargs['dropout']
        self.norm_tool = GraphNormTool(kwargs['adj_norm'], 'gcn')

    def forward(self, x, adj, node_selector = None, labels = None, avg_loss = True):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout_rate, training=self.training)
        x = self.gc2(x, adj)        
        logits = F.log_softmax(x, dim=1)

        if node_selector is not None:
            logits = logits[node_selector]
        
        if labels is not None:
            if node_selector is not None:
                labels = labels[node_selector]
            loss = F.nll_loss(logits, labels, reduce=avg_loss)
            pred = logits.data.max(1, keepdim=True)[1]
            acc = pred.eq(labels.data.view_as(pred)).cpu()
            return pred, loss, acc
        else:
            return pred

In [79]:
def glorot_uniform(t):
    if len(t.size()) == 2:
        fan_in, fan_out = t.size()
    elif len(t.size()) == 3:
        # out_ch, in_ch, kernel for Conv 1
        fan_in = t.size()[1] * t.size()[2]
        fan_out = t.size()[0] * t.size()[2]
    else:
        fan_in = np.prod(t.size())
        fan_out = np.prod(t.size())

    limit = np.sqrt(6.0 / (fan_in + fan_out))
    t.uniform_(-limit, limit)

def _param_init(m):
    if isinstance(m, Parameter):
        glorot_uniform(m.data)
    elif isinstance(m, nn.Linear):
        m.bias.data.zero_()
        glorot_uniform(m.weight.data)

def weights_init(m):
    for p in m.modules():
        if isinstance(p, nn.ParameterList):
            for pp in p:
                _param_init(pp)
        else:
            _param_init(p)

    for name, p in m.named_parameters():
        if not '.' in name: # top-level parameters
            _param_init(p)

In [83]:
class GraphNormTool(object):
    def __init__(self, adj_norm, gm):
        self.adj_norm = adj_norm
        self.gm = gm
        g = StaticGraph.graph

        edges = np.array(g.edges(), dtype=np.int64)
        rev_edges = np.array([edges[:, 1], edges[:, 0]], dtype=np.int64)
        self_edges = np.array([range(len(g)), range(len(g))], dtype=np.int64)

        edges = np.hstack((edges.T, rev_edges, self_edges))

        idxes = torch.LongTensor(edges)
        values = torch.ones(idxes.size()[1])

        self.raw_adj = torch.sparse.FloatTensor(idxes, values, StaticGraph.get_gsize())
        if ctx == 'gpu':
            self.raw_adj = self.raw_adj.cuda()
        
        self.normed_adj = self.raw_adj.clone()
        if self.adj_norm:
            if self.gm == 'gcn':
                GraphLaplacianNorm(self.normed_adj)
            else:
                GraphDegreeNorm(self.normed_adj)

    def norm_extra(self, added_adj = None):
        if added_adj is None:
            return self.normed_adj

        new_adj = self.raw_adj + added_adj
        if self.adj_norm:
            if self.gm == 'gcn':
                GraphLaplacianNorm(new_adj)
            else:
                GraphDegreeNorm(new_adj)
        return new_adj


In [85]:
def GraphLaplacianNorm(raw_adj):
    ones = torch.ones(raw_adj.size()[0], 1)
    if raw_adj.is_cuda:
        ones = ones.cuda()
    norm = torch.mm(raw_adj, ones) ** 0.5
    indices = raw_adj._indices()
    values = raw_adj._values()
    if not values.is_cuda:
        my_lib.graph_laplacian_norm(indices, values, norm)
    else:
        my_lib.graph_laplacian_norm_cuda(indices, values, norm)

In [86]:
base_model = load_base_model()
base_model

NameError: name 'my_lib' is not defined

In [23]:
def init_setup():
    features, labels, _, idx_val, idx_test = load_txt_data(cmd_args.data_folder + '/' + cmd_args.dataset, cmd_args.dataset)    
    features = Variable( features )
    labels = Variable( torch.LongTensor( np.argmax(labels, axis=1) ) )
    if cmd_args.ctx == 'gpu':
        labels = labels.cuda()

    base_model = load_base_model() # load base model
    run_test(base_model, features, Variable( base_model.norm_tool.normed_adj ), idx_test, labels)

    dict_of_lists = load_raw_graph(cmd_args.data_folder + '/' + cmd_args.dataset, cmd_args.dataset)

    return features, labels, idx_val, idx_test, base_model, dict_of_lists